# TP2: Seafloor classification par CNN 

## 1 - Introduction

Disposant d'un ensemble d'images dont on veut prédire la classe, deux possibilités s'offrent pour apprendre un modèle profond de classement.

- La première possibilité se nomme "Transfer Learning" associé au "fine tuning" dont les principes sont d'utiliser un réseau de neurones profond entrainé dans un autre contexte et de l'adapter à nos données: 
    
- La seconde possibilité est de créer et d'entrainer un modèle profond ex-nihilo (from scratch, en partant de zéro).


L'objectif de ce TP est d'appliquer ces deux possibilités au problème de classification de patchs d'images sonar en types de fond marin que vous avez déjà traités dans les TPs précédent. Vous reprendrez les fonctions d'import des patchs que vous avez déjà mises au point lors des tps précédents.

Il pourra être néanmoins utile d'utiliser les lignes de code suivantes pour que les données soient dans la forme attendue par tensoflow/keras et éventuellement changer la taille des images:
```python
from keras.preprocessing.image import img_to_array, load_img
target_size = 192;
feature_values = np.array([img_to_array(load_img(img,  # color_mode = "grayscale",
             target_size=(target_size, target_size))) for img in dataset_df['image_path'].values.tolist()
]).astype('float32')
```
A noter que la taille des patchs est automatiquement réduite à 192x192 pixels, vous pouvez réduire cette taille en fonction des performances de votre machine. Attention, une taille trop basse peut impliquer une erreur si vous réutilisez par la suite un modèle déjà entrainé (par exemple VGG16).

La création d'ensemble d'apprentissage, de validation et de test se fera en divisant la base en trois parts. Il faudrait pour ce petit jeu de données réaliser une procédure de cross-validation. Compte tenu du temps pour réaliser cette procédure, elle sera ici laissée de coté. 

## 1 - Tutorials CNN et transfer learning par fine tuning

Dans le startercode, vous trouverez un jupyter notebooks qui vous servira de base pour réaliser la suite. Dans un premier temps, le notebook détaille le fonctionnement et la mise en oeuvre des CNNs (en particulier les différentes couches d'un CNN avec des exemples); pour ensuite détailler la procédure liée au fine tuning avec data augmentation à partir de données de type TensorFlow Dataset. 

Veillez à bien suivre les différentes étapes et à bien comprendre les différentes commandes employées. Si vous voulez d'autres exemples, vous trouverez d'autres ressources supplémentaires.


## 2 - Transfer learning par fine tuning sur le dataset seafloor

- Vous commencerez par le fine tuning en vous inspirant du tuto fourni ci-dessus pour faire du transfer learning du modèle xception (dont les paramètres ont été appris sur la base d'images "imageNet") pour l'appliquer aux patchs d'images sonar. Vous procéderez ainsi:
  - les modèles sont téléchargeables ici si vous avez des problèmes pour télécharger: https://drive.google.com/open?id=1qFwqoNU1fsvl8fu-7eRCmjoPNZZNzPSJ
  - Résumer l'approche du transfer learning/fine tuning

L'idée du transfert learning est d'utiliser un modèle déjà entrainé par d'autres personnes ayant des moyens matériels plus importants que notre pc pour entrainer le modèle. Par la suite, on réutilise les possibilités offertes par ce réseau de neurones en terme d'extraction de features afin d'alimenter les dernières couches de notre modèle propre à notre problème, ici de classification. Il ne nous reste que ces dernières couches à entrainer afin d'avoir un modèle opérationnel.

  - Décriver l'architecture du modèle utilisé (xception ici)

L'idée des réseaux inceptions dont xception fait parti est de réaliser en parallèle 3 traitements de convolution 1x1 suivis d'une convolution 3x3 et de concaténer les sorties de ces trois branches pour former un block. Ensuite ces blocks sont chainés afin d'être réutilisés plusieurs fois et de pouvoir avoir un modèle plus complexe. Cette architecture offre des résultats assez satisfaisants.

  - Vous précisérez votre choix concernant les paramètres des fonctions appelées en particulier expliquer votre démarche concernant les phases de preprocessing des images, de data augmentation, de classification, etc.
  - remarque: comme les images sonar sont en niveaux de gris et que le modèle VGG prend en entrée des images couleurs, il s'agira de dupliquer ce canal sur les canaux R, G et B. 

  - Enfin, vous évaluerez les performances obtenues.
- (Bonus) comparez les résultats obtenus par l'architecture vgg16 (https://keras.io/applications/).

- (Bonus) Essayez et comparez les résultats obtenus par d'autres architectures (Resnet, Inception etc...https://keras.io/applications/).

## 3 - Proposition de votre propre achitecture  

- Vous proposerez ensuite une architecture de réseau profond convolutif et évaluerez ses performances. 
- Expliquez brièvement votre architecture et en particulier à quoi servent les couches (et leur enchainement) de votre architecture.
- Vous comparerez ensuite les performances obtenus (par rapport à ceux obtenus à la partie précédente) sur la matrice de confusion et les métriques de performance classiques.






# A rendre 
- pour le **12/01/21**
- la séance du 05/01/21 sera consacrée à finaliser
- **Commenter au maximum votre code (pourquoi vous utilisez tel ou tel bout de code) ou apporter des précisions dans votre CR.**
- au choix (**N'oublier pas les deux noms en cas de binômes**):
    - un fichier zip avec *.py et un cr au format pdf
    - un fichier .ipynb avec compte-rendu et code



# 4 - Aide pour démarrer

## 4.1 Chargement des données
Vous pourrez utiliser cette procédure pour charger les données: 
**A noter**
- target_size permet de définir un éventuel changement de taille des images qui pourra servir en fonction de la taille d'entrée du modèle que vous considérez.
- comme les images sonar sont en niveaux de gris et que le modèle VGG prend en entrée des images couleurs, load_img duplique ce canal sur les canaux R, G et B.

In [1]:
import tensorflow.keras as keras
import pandas as pd
import os 
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator, img_to_array, load_img
from sklearn import preprocessing


import sys
IN_COLAB = 'google.colab' in sys.modules

# Paramètres
if IN_COLAB:
    from google.colab import drive
    drive.mount('/content/drive', force_remount=True)
    os.chdir("/content/drive/MyDrive/Machine_Learning")
    DATASET_PATH = "./dataset/imgs/"
    LABEL_PATH = "./dataset/labels/labels.csv"
    from pythonTools import *
else:
    IN_COLAB = False
    DATASET_PATH = r'./dataset/imgs/'
    LABEL_PATH = r'./dataset/labels/labels.csv'

target_size = 200
dataset_df = pd.read_csv(LABEL_PATH)

# We add another column to the labels dataset to identify image path
dataset_df['image_path'] = dataset_df.apply(lambda row: (DATASET_PATH + row["id"]), axis=1)

batch_imgs = np.array([img_to_array(
    load_img(img, color_mode = "grayscale",
             target_size=(target_size, target_size))
) for img
    in dataset_df['image_path'].values.tolist()
]).astype('float32')

#  Noms des labels dans l'ordre, respectivement aux indices
labelNames_unique = np.array(["Posidonia","Ripple 45°","Ripple vertical","Rock","Sand","Silt"])
labelDict={}
for i in range(len(labelNames_unique)):
    labelDict.update({i:labelNames_unique[i]})

# Récupération des labels
label_names = dataset_df['seafloor']

# nb de classes
label_nb = labelNames_unique.shape[0]

# indices
le = preprocessing.LabelEncoder()
le.fit(labelNames_unique)
labelIndices_unique = le.transform(labelNames_unique)
labelIndices  = le.transform(label_names)

# one-hot-encoding
labelOhe = pd.get_dummies(label_names.reset_index(drop=True)).values

Mounted at /content/drive
None


In [2]:
print(batch_imgs.shape)

(360, 200, 200, 1)


## 4.2 - Définition, entrainer et évaleur le modèle VGG16 (par exemple) pour le fine tuning
Ensuite procéder comme le tuto:
- charger le modèle VGG16 sans le classifieur include_top=False
- Visualiser l'architecture du modèle: model.summary()
- Créer un modèle d'extraction d'information (features) allant de la couche d'entrée de VGG16 jusqu'à sa dernière couche de convolution nommée 'block3_pool'
- Rajouter des couches Dense pour définir un classifieur Fully connected (attention aux nombres de sorties de la dernière couche et à sa fonction d'activation)
- ne pas oublier l'étape de preprocessing essentiel à la bonne réussite de l'apprentissage. Il faut se renseigner sur les corrections à apporter et la taille des images d'entrée
- Compiler (compile), entrainer (fit) et évaluer (evaluate) le modèle


In [3]:
# Importing the required libraries
import tensorflow as tf
import keras
import pandas as pd
import numpy as np
from keras.models import Sequential
from keras.layers import Conv2D, MaxPool2D, Flatten, Dense
from keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing import image
from sklearn.model_selection import train_test_split

In [4]:
# Spliting data into a training and a testing dataset
data_train, data_test, labels_train, labels_test = train_test_split(batch_imgs, label_names, test_size=0.20, random_state=42)
print(data_train.shape, data_test.shape, labels_train.shape, labels_test.shape)
labels_train = le.transform(labels_train)
labels_test = le.transform(labels_test)

(288, 200, 200, 1) (72, 200, 200, 1) (288,) (72,)


In [5]:
# The CNN expects 224 × 224 images, so we need
# to resize them. We also need to run the image through Xception’s preprocess_input() function:
    
def preprocess(image):
    resized_image = tf.image.resize(image, [224, 224])
    final_image = keras.applications.xception.preprocess_input(resized_image)
    return final_image

In [6]:
batch_size = 32

train_set = []
for k in range(len(data_train)):
  img = preprocess(data_train[i])
  img=np.stack((img[:,:,0],)*3,axis=-1)
  img=tf.Variable(img,trainable=True)
  train_set.append(img)

test_set = []
for k in range(len(data_test)):
  img = preprocess(data_test[i])
  img=np.stack((img[:,:,0],)*3,axis=-1)
  img=tf.Variable(img,trainable=True)
  test_set.append(img)
    
# Creating a StandardScaler()
scaler = preprocessing.StandardScaler()

# Getting statistical indicators from the dataset and normalizing training and testing data

train_set=tf.convert_to_tensor(train_set)
test_set=tf.convert_to_tensor(test_set)


print(train_set.shape,test_set.shape)

(288, 224, 224, 3) (72, 224, 224, 3)


In [16]:
# chargement du feature extractor (sans classifieur include_top=False)
base_model = keras.applications.xception.Xception(weights="imagenet",
                                                  include_top=False)

# on rajoute séquentiellement des couches de classification
n_classes = 6
avg = keras.layers.GlobalAveragePooling2D()(base_model.output)
output = keras.layers.Dense(n_classes, activation="softmax")(avg)

# on a recréé le modèle en liant l'entrée de base_model à la sortie 
model = keras.models.Model(inputs=base_model.input, outputs=output)

In [17]:
model.summary()

Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            [(None, None, None,  0                                            
__________________________________________________________________________________________________
block1_conv1 (Conv2D)           (None, None, None, 3 864         input_3[0][0]                    
__________________________________________________________________________________________________
block1_conv1_bn (BatchNormaliza (None, None, None, 3 128         block1_conv1[0][0]               
__________________________________________________________________________________________________
block1_conv1_act (Activation)   (None, None, None, 3 0           block1_conv1_bn[0][0]            
____________________________________________________________________________________________

In [20]:
# Principe du fine-tuning = on fixe le feature extractor pendant l'apprentissage
for layer in base_model.layers:
    layer.trainable = False

# Since our model uses the base model’s layers directly, rather than the base_model object itself 
# (i.e. on a recréé le modèle à partir des layers avec l'appel à keras.models.Model), 
# setting base_model.trainable=False
# would have no effect.

# apprentissage
optimizer = keras.optimizers.SGD(lr=0.001, momentum=0.9, decay=0.01)
model.compile(loss="sparse_categorical_crossentropy", optimizer=optimizer,
              metrics=["accuracy"])
dataset_size = 360
history = model.fit(tf.convert_to_tensor(train_set), labels_train, epochs=50, 
                    validation_data=(tf.convert_to_tensor(test_set), labels_test))

Epoch 1/50
9/9 [==============================] - 4s 280ms/step - loss: 1.7943 - accuracy: 0.1515 - val_loss: 1.8015 - val_accuracy: 0.1944
Epoch 2/50
9/9 [==============================] - 2s 189ms/step - loss: 1.7897 - accuracy: 0.1728 - val_loss: 1.8018 - val_accuracy: 0.1944
Epoch 3/50
9/9 [==============================] - 2s 191ms/step - loss: 1.7855 - accuracy: 0.2137 - val_loss: 1.8022 - val_accuracy: 0.1944
Epoch 4/50
9/9 [==============================] - 2s 189ms/step - loss: 1.7878 - accuracy: 0.1974 - val_loss: 1.8018 - val_accuracy: 0.1944
Epoch 5/50
9/9 [==============================] - 2s 191ms/step - loss: 1.7941 - accuracy: 0.1397 - val_loss: 1.8011 - val_accuracy: 0.1944
Epoch 6/50
9/9 [==============================] - 2s 190ms/step - loss: 1.7902 - accuracy: 0.1568 - val_loss: 1.8015 - val_accuracy: 0.1944
Epoch 7/50
9/9 [==============================] - 2s 190ms/step - loss: 1.7921 - accuracy: 0.1747 - val_loss: 1.8019 - val_accuracy: 0.1944
Epoch 8/50
9/9 [====

In [21]:
for layer in base_model.layers:
    layer.trainable = True

optimizer = keras.optimizers.SGD(learning_rate=0.01, momentum=0.9,
                                 nesterov=True, decay=0.001)
model.compile(loss="sparse_categorical_crossentropy", optimizer=optimizer,
              metrics=["accuracy"])
dataset_size = 360
history = model.fit(tf.convert_to_tensor(train_set), labels_train, epochs=20, 
                    validation_data=(tf.convert_to_tensor(test_set), labels_test))


Epoch 1/20
9/9 [==============================] - 10s 743ms/step - loss: 1.8013 - accuracy: 0.2018 - val_loss: 1.7935 - val_accuracy: 0.1944
Epoch 2/20
9/9 [==============================] - 6s 671ms/step - loss: 1.7923 - accuracy: 0.1601 - val_loss: 1.8008 - val_accuracy: 0.2083
Epoch 3/20
9/9 [==============================] - 6s 674ms/step - loss: 1.7994 - accuracy: 0.1662 - val_loss: 1.8002 - val_accuracy: 0.0972
Epoch 4/20
9/9 [==============================] - 6s 685ms/step - loss: 1.7900 - accuracy: 0.2006 - val_loss: 1.8012 - val_accuracy: 0.0972
Epoch 5/20
9/9 [==============================] - 6s 688ms/step - loss: 1.7893 - accuracy: 0.2183 - val_loss: 1.8006 - val_accuracy: 0.0972
Epoch 6/20
9/9 [==============================] - 6s 677ms/step - loss: 1.7885 - accuracy: 0.1816 - val_loss: 1.7961 - val_accuracy: 0.0972
Epoch 7/20
9/9 [==============================] - 6s 672ms/step - loss: 1.7879 - accuracy: 0.1896 - val_loss: 1.7939 - val_accuracy: 0.0972
Epoch 8/20
9/9 [===

## 4.3 - Custom CNN


In [57]:
from sklearn.preprocessing import StandardScaler

X_train, X_test, y_train, y_test = train_test_split(batch_imgs, labelIndices, test_size = 0.25, random_state = 0)

X_train /= 255.
X_test /= 255.

x_tr = []
for m in X_train:
  x_tr.append(m.flatten())

x_te = []
for m in X_test:
  x_te.append(m.flatten())

X_train, X_test = np.asarray(x_tr), np.asarray(x_te)
print(X_train.shape, y_train.shape, X_test.shape, y_test.shape)

(270, 40000) (270,) (90, 40000) (90,)


In [60]:
from keras.models import Sequential
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense 
from keras.layers import Dropout

from sklearn.model_selection import GridSearchCV
from keras.wrappers.scikit_learn import KerasClassifier

#Fonction de construction du CNN
def build_classifier(optimizer='adam'):
    # Initialising the CNN
    classifier = Sequential()

    # Step 4 - Full connection
    classifier.add(Dense(units = 128, activation = 'relu',input_dim = 40000))
    classifier.add(Dense(units = 128, activation = 'relu'))
    #classifier.add(Dense(units = 128, activation = 'relu'))

    classifier.add(Dense(units = 6, activation = 'softmax'))

    # Compiling the CNN
    classifier.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])
    return classifier

#Instance du classifier
classifier = KerasClassifier(build_fn = build_classifier)

parameters = {'batch_size': [30, 60],
              'epochs': [5, 10, 15],
              'optimizer': ['adam']}
#Création de la grille d'entraînement.
grid_search = GridSearchCV(estimator = classifier,
                           param_grid = parameters,
                           scoring = 'accuracy',
                           cv = 5)

grid_search = grid_search.fit(X_train, y_train, verbose=0)
best_parameters = grid_search.best_params_
best_accuracy = grid_search.best_score_
print(best_accuracy,best_parameters)
score = grid_search.score(X_test, y_test)
print(score)

0.1814814814814815 {'batch_size': 30, 'epochs': 5, 'optimizer': 'adam'}
0.17777777777777778


On remarque que les différents résultats obtenus ne sont pas à la hauteur de mes attentes. On obtient environ une accuracy de 20% ce qui n'est pas exceptionnel. On remarque cependant que les deux méthodes mennent aux mêmes résultats. J'en conclus donc que le problème vient sûrement de la mise en place du dataset. C'est quelque chose qui m'a paru assez difficile et peu trouvable sur internet puisque la plupart des dataset sont déjà formés et téléchargeables en une ligne de commande. La mise en forme de ce dataset me semble assez capricieuse et stricte.

# 5 -  Ressources supplémentaires en transfer learning par fine tuning**
 En dehors des supports de cours, vous pourrez aussi vous appuyer sur:
- Les concepts du transfer learning sont expliqués dans les liens ci-dessous:
  - https://www.youtube.com/watch?v=FQM13HkEfBk&index=20&list=PLkDaE6sCZn6Gl29AoE31iwdVwSG-KnDzF
  - http://cs231n.github.io/transfer-learning/
  - https://flyyufelix.github.io/2016/10/03/fine-tuning-in-keras-part1.html et https://flyyufelix.github.io/2016/10/08/fine-tuning-in-keras-part2.html
- Des exemples supplémentaires d'implémentation
  - https://github.com/dipanjanS/hands-on-transfer-learning-with-python/blob/master/notebooks/Ch06%20-%20Image%20Recognition%20and%20Classification/CIFAR10_CNN_Classifier.ipynb et https://github.com/dipanjanS/hands-on-transfer-learning-with-python/blob/master/notebooks/Ch06%20-%20Image%20Recognition%20and%20Classification/CIFAR10_VGG16_Transfer_Learning_Classifier.ipynb
  - https://github.com/dipanjanS/hands-on-transfer-learning-with-python/blob/master/notebooks/Ch06%20-%20Image%20Recognition%20and%20Classification/Dog_Breed_EDA.ipynb et https://github.com/dipanjanS/hands-on-transfer-learning-with-python/blob/master/notebooks/Ch06%20-%20Image%20Recognition%20and%20Classification/Dog_Breed_Transfer_Learning_Classifier.ipynb
- cours et des vidéos de Stanford University: https://www.youtube.com/watch?v=wEoyxE0GP2M&list=PL3FW7Lu3i5JvHM8ljYj-zLfQRF3EO8sYv&index=6, https://www.youtube.com/watch?v=wEoyxE0GP2M&list=PL3FW7Lu3i5JvHM8ljYj-zLfQRF3EO8sYv&index=7)
 